# Jagged arrays and tables

This notebook demonstrates how Numpy idioms have been extended for jagged (i.e. "ragged") arrays, which are nested arrays that do not have a fixed number of elements in each subarray.

`JaggedArray` and `Table` are both represented as columns in memory: like-attributes are contiguous in memory and unlike-attributes may be anywhere in memory. The data structures are designed to compose, allowing for multiple levels of jaggedness or nested tables.

In [2]:
import os
os.chdir(os.path.expanduser("~"))

import pandas
import numpy
from awkward import JaggedArray, Table

Let's start with a simple jagged array: it contains four subarrays, whose lengths are 3, 0, 2, and 5.

In [3]:
a = JaggedArray.fromiter([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5, 6.6, 7.7, 8.8, 9.9]])
a

<JaggedArray [[0.  1.1 2.2] [] [3.3 4.4] [5.5 6.6 7.7 8.8 9.9]] at 7167be2f66d0>

Although it is presented above as a list of lists, it consists of three Numpy arrays: `starts` (indexes where each starts), `stops` (index where each stops), and `content` (the data themselves, may be any type).

In [4]:
print(a.starts)
print(a.stops)
print(a.content)

[0 3 3 5]
[ 3  3  5 10]
[0.  1.1 2.2 3.3 4.4 5.5 6.6 7.7 8.8 9.9]


With this data representation, we can take advantage of Numpy for fast calculations. Numpy ufuncs (such as this addition by a constant, below) are passed to the `content` array. The `starts` and `stops` are unchanged and therefore shared.

In [5]:
a + 100

<JaggedArray [[100.  101.1 102.2] [] [103.3 104.4] [105.5 106.6 107.7 108.8 109.9]] at 7167edba8f10>

But since jagged arrays have more structure than Numpy arrays, we have to extend the broadcasting rules. A jagged array plus a flat array adds each jagged subarray with the corresponding scalar from the flat array.

In [6]:
a + numpy.array([100, 200, 300, 400])

<JaggedArray [[100.  101.1 102.2] [] [303.3 304.4] [405.5 406.6 407.7 408.8 409.9]] at 7167be2f6650>

Just as with broadcasting in Numpy, the length of the jagged array (4) should match the length of the flat array (5 in the case below). The following should fail.

In [7]:
a + numpy.array([100, 200, 300, 400, 500])

ValueError: cannot broadcast JaggedArray of shape (4,) with Numpy array of shape (5,)

Additionally, slices of jagged arrays should act on the subarrays as elements.

In [8]:
a[::2]

<JaggedArray [[0.  1.1 2.2] [3.3 4.4]] at 7167be259f50>

And multidimensional indexing should work as it does in Numpy. In the case below, we select the first and third subarray (first dimension) and then the second element from each (second dimension). Unlike Numpy, some subarrays might have a second element while others don't. All of the requisite checks are implemented without Python for loops.

In [9]:
a[::2, 1]

array([1.1, 4.4])

## Nested jagged arrays

If we want subarrays within subarrays, we can make one `JaggedArray` the `content` of another. All of the jagged indexing rules apply recursively.

In [10]:
b = JaggedArray.fromoffsets([0, 2, 4], a)
b

<JaggedArray [[[0.  1.1 2.2] []] [[3.3 4.4] [5.5 6.6 7.7 8.8 9.9]]] at 7167be217190>

Since this jagged-jagged array is three dimensional, we can pass up to three integers/slices/masks/fancy indexes in its square brackets.

Below, we select the second item (`[[3.3, 4.4], [5.5, 6.6, 7.7, 8.8, 9.9]]`), then take all subitems, then take the second subsubitem from each (`4.4` from the first and `6.6` from the second).

In [11]:
b[1, :, 1]

array([4.4, 6.6])

Sometimes, it's easier to see the structure by converting to Python lists (for small objects).

In [12]:
a.tolist()

[[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5, 6.6, 7.7, 8.8, 9.9]]

In [13]:
b.tolist()

[[[0.0, 1.1, 2.2], []], [[3.3, 4.4], [5.5, 6.6, 7.7, 8.8, 9.9]]]

It can also help to view them as Pandas DataFrames. In Pandas, we represent the nested structure with a MultiIndex: the first key column is the outer index (`0`, `2`, `3`, skipping the empty one) and the second key column is the inner index.

In [14]:
a.pandas()

0
0 0  0.0
  1  1.1
  2  2.2
2 0  3.3
  1  4.4
3 0  5.5
  1  6.6
  2  7.7
  3  8.8
  4  9.9

With a jagged-jagged array, the MultiIndex gets three columns.

In [15]:
b.pandas()

0
0 0 0  0.0
    1  1.1
    2  2.2
1 0 0  3.3
    1  4.4
  1 0  5.5
    1  6.6
    2  7.7
    3  8.8
    4  9.9

These views and some of the internal calculations are made possibly by switching representations. A jagged array expressed by `starts` and `stops` can be expressed by a `parents` array: one element per `content` element, specifying the outer array (i.e. the "parent") the content belongs to.

The representations have different strengths and weaknesses: `parents` allows reduction operations to be vectorized, but it can only express empty lists by omission. In particular, we'll never know if there are any empty subarrays after the last element!

In [16]:
a.parents

array([0, 0, 0, 2, 2, 3, 3, 3, 3, 3])

In [17]:
a.index

<JaggedArray [[0 1 2] [] [0 1] [0 1 2 3 4]] at 7167be259150>

In the [array types language](https://mybinder.org/v2/gh/scikit-hep/awkward-array/0.0.4?filepath=binder%2Farray-types.ipynb), the jagged and the jagged-jagged arrays differ by one index:

In [18]:
print(a.type)
print(b.type)

[0, 4) -> [0, inf) -> float64
[0, 2) -> [0, inf) -> [0, inf) -> float64


Continuing with our tour of how jagged arrays extend Numpy, consider reductions: in Numpy, they turn flat arrays into scalars. They should turn jagged arrays into flat arrays.

In [19]:
a.sum()

array([ 3.3,  0. ,  7.7, 38.5])

For maximization and minimization, we need an identity element for empty lists. Infinity is a natural identity for minimization, and negative infinity is a natural identity for maximization.

In [20]:
a.max()

array([ 2.2, -inf,  4.4,  9.9])

When computing the `argmax` (index at which the maximum appears), it's better to return a singleton or an empty list, represented as jagged arrays.

In [21]:
a.argmax()

<JaggedArray [[2] [] [1] [4]] at 7167be1c8150>

That way, the `argmax` can be used as a fancy index for a jagged array of the same structure (just as `argmax` is typically used in Numpy).

In [22]:
a[a.argmax()]

<JaggedArray [[2.2] [] [4.4] [9.9]] at 7167be2f6550>

If we don't care which subarrays were empty, we can `flatten` the result. Using this `JaggedArray` data structure, list-flattening is very fast because the flattened result is usually just the `content` (when `starts` and `stops` are contiguous, which they usually are).

In [23]:
a[a.argmax()].flatten()

array([2.2, 4.4, 9.9])

For completeness, note that we can use jagged masks just as we can use jagged fancy indexes. The results are what you'd expect.

In [24]:
a > 5

<JaggedArray [[False False False] [] [False False] [ True  True  True  True  True]] at 7167be259d10>

In [25]:
a[a > 5]

<JaggedArray [[] [] [] [5.5 6.6 7.7 8.8 9.9]] at 7167be1c8410>

In [26]:
a[a > 5].sum()

array([ 0. ,  0. ,  0. , 38.5])

## Tables

Arrays, including jagged arrays, map integer indexes to data, and a table row maps string-valued column names to data.

It may not be clear at first why we need a `Table` type— after all, Numpy has structured arrays and Pandas has DataFrames. We need it for three reasons:

   * `Table` and `JaggedArray` know about each other: they can be nested within each other.
   * Numpy's structured arrays are contiguous in memory and row-wise: different columns in the same row are adjacent in memory. `Table`, however, is columnar: different rows in the same column are adjacent in memory, and columns need not have any locality relationship to each other.
   * Pandas DataFrames have their own memory management. We need more control.

Conceptually, a `Table` resembles a `zip` in Python:

In [27]:
c = Table.zip(one=[0, 1, 2, 3, 4], two=[0.0, 1.1, 2.2, 3.3, 4.4])
c

<Table 5 x 2 at 7167be1c8990>

The contents are suppressed because they may be very large. If not, you can call `.tolist()` on them, like any other array. However, these "arrays" are represented as a `list` of `dicts`.

In [28]:
c.tolist()

[{'one': 0, 'two': 0.0},
 {'one': 1, 'two': 1.1},
 {'one': 2, 'two': 2.2},
 {'one': 3, 'two': 3.3},
 {'one': 4, 'two': 4.4}]

It's also convenient to view the `Table` as a Pandas `DataFrame`.

In [29]:
c.pandas()

,two,one
0,0.0,0
1,1.1,1
2,2.2,2
3,3.3,3
4,4.4,4


Although the columnar representation motivated the design of `Tables`, Numpy structured arrays (a.k.a. "recarrays") can be viewed as— not copied into— a `Table`. This is because the `Table` constructor only cares that its columns are arrays; columns of a Numpy structured array are Numpy arrays with an unusually large stride.

In [30]:
d = Table.fromrec(numpy.array([(0, 0.0), (1, 1.1), (2, 2.2), (3, 3.3), (4, 4.4)], dtype=[("one", int), ("two", float)]))
d

<Table 5 x 2 at 7167be1c8e10>

In [31]:
d.tolist()

[{'one': 0, 'two': 0.0},
 {'one': 1, 'two': 1.1},
 {'one': 2, 'two': 2.2},
 {'one': 3, 'two': 3.3},
 {'one': 4, 'two': 4.4}]

In [32]:
d.pandas()

,one,two
0,0,0.0
1,1,1.1
2,2,2.2
3,3,3.3
4,4,4.4


One important aspect of `Table` indexing, inherited from Numpy, is that string-valued column indexes commute with integer-valued row indexes.

If you put a string in square brackets first, you get an array.

In [33]:
c["two"], d["two"]

(array([0. , 1.1, 2.2, 3.3, 4.4]), array([0. , 1.1, 2.2, 3.3, 4.4]))

If you put an integer (or slice, mask, fancy index) in square brackets first, you get a row.

In [34]:
c[2], d[2]

(<Table.Row 2>, <Table.Row 2>)

In [35]:
c[2].tolist(), d[2].tolist()

({'one': 2, 'two': 2.2}, {'one': 2, 'two': 2.2})

But whichever order you do it in, you will eventually get to a table cell.

In [36]:
c[2]["two"], d[2]["two"]

(2.2, 2.2)

In [37]:
c[2].two, d[2].two

(2.2, 2.2)

Like jagged arrays, `Tables` respond to Numpy ufuncs, applying the appropriate transformation. In the case below, we add `100` to all rows and columns of a table and immediately view it as Pandas.

(Pandas has such a nice screen representation for small tables.)

In [38]:
(c + 100).pandas()

,two,one
0,100.0,100
1,101.1,101
2,102.2,102
3,103.3,103
4,104.4,104


In the language of array types, a `Table` is indexed by an integer and a string from an enumeration set.

In [39]:
print(c.type)

[0, 5) -> 'two' -> float64
          'one' -> int64


Just as `Table.zip` creates a flat table, `JaggedArray.zip` creates a jagged table: a `Table` inside a `JaggedArray`.

In [40]:
e = JaggedArray.zip(one=a, two=a*2)
e

<JaggedArray [[<Table.Row 0> <Table.Row 1> <Table.Row 2>] [] [<Table.Row 3> <Table.Row 4>] [<Table.Row 5> <Table.Row 6> <Table.Row 7> <Table.Row 8> <Table.Row 9>]] at 7167be1c8e50>

In [41]:
e.pandas()

two  one
0 0   0.0  0.0
  1   2.2  1.1
  2   4.4  2.2
2 0   6.6  3.3
  1   8.8  4.4
3 0  11.0  5.5
  1  13.2  6.6
  2  15.4  7.7
  3  17.6  8.8
  4  19.8  9.9

Columnar projections of this jagged table are jagged arrays.

In [42]:
e["two"].tolist()

[[0.0, 2.2, 4.4], [], [6.6, 8.8], [11.0, 13.2, 15.4, 17.6, 19.8]]

Rows are subtables, just as the elements of a jagged array are subarrays.

In [43]:
e[2].tolist()

[{'one': 3.3, 'two': 6.6}, {'one': 4.4, 'two': 8.8}]

In [44]:
e[2]["two"].tolist()

[6.6, 8.8]

In [45]:
e[::2, 1].tolist()

[{'one': 1.1, 'two': 2.2}, {'one': 4.4, 'two': 8.8}]

In the language of array types, this is indexed by two integers— the first definite, the second indefinite— followed by an enumerated string.

In [46]:
print(e.type)

[0, 4) -> [0, inf) -> 'two' -> float64
                      'one' -> float64


The combination of jaggedness and tables allows for a great deal of expressivity. In this last example, we take a per-subarray cross-join of two jagged arrays, expressing the 2-tuples of all combinations as a jagged table.

In [47]:
f = JaggedArray.fromiter([[True, False], [True, False], [True, False], [True, False]])
g = a.cross(f)
g

<JaggedArray [[<Table.Row 0> <Table.Row 1> <Table.Row 2> <Table.Row 3> <Table.Row 4> <Table.Row 5>] [] [<Table.Row 6> <Table.Row 7> <Table.Row 8> <Table.Row 9>] [<Table.Row 10> <Table.Row 11> <Table.Row 12> ... <Table.Row 17>, <Table.Row 18>, <Table.Row 19>]] at 7167be1de410>

In [48]:
g.pandas()

0      1
0 0  0.0   True
  1  0.0  False
  2  1.1   True
  3  1.1  False
  4  2.2   True
  5  2.2  False
2 0  3.3   True
  1  3.3  False
  2  4.4   True
  3  4.4  False
3 0  5.5   True
  1  5.5  False
  2  6.6   True
  3  6.6  False
  4  7.7   True
  5  7.7  False
  6  8.8   True
  7  8.8  False
  8  9.9   True
  9  9.9  False

This is a particularly useful operation for particle physics: we routinely need to iterate over all combinations of particles from the same event (sublist). This cross-join lets us perform calculations over pairs of particles (e.g. invariant mass) without for loops, allowing for GPU vectorization and SQL-style expressivity.

The for loop below is equivalent to the per-subarray cross-join above.

In [49]:
for x, y in zip(a, f):
    print("new sublist")
    for xi in x:
        for yj in y:
            print("    {} and {}".format(xi, yj))

new sublist
    0.0 and True
    0.0 and False
    1.1 and True
    1.1 and False
    2.2 and True
    2.2 and False
new sublist
new sublist
    3.3 and True
    3.3 and False
    4.4 and True
    4.4 and False
new sublist
    5.5 and True
    5.5 and False
    6.6 and True
    6.6 and False
    7.7 and True
    7.7 and False
    8.8 and True
    8.8 and False
    9.9 and True
    9.9 and False
